In [ ]:
import numpy as np
import plotly.graph_objects as go

# Simulated structure for demo purposes (we'll generate a circular mesh)
# Let's create a synthetic setup with a few radial lines and nodes

class Point:
    def __init__(self, x, y, z=0):
        self.x = x
        self.y = y
        self.z = z

    def __repr__(self):
        return f"Point({self.x:.2f}, {self.y:.2f}, {self.z:.2f})"

class Line:
    def __init__(self, start, end):
        self.start = start
        self.end = end

# Settings for demo generation
num_radial = 12  # 12 radial lines
ring_divs = [5 + (i % 3) for i in range(num_radial)]  # Varying divisions

circle_radius = 1
rectangle_radius = 2

radial_lines_dict = {}

# Generate radial lines
for i in range(num_radial):
    angle = 2 * np.pi * i / num_radial
    inner = Point(circle_radius * np.sin(angle), circle_radius * np.cos(angle), 0)
    outer = Point(rectangle_radius * np.sin(angle), rectangle_radius * np.cos(angle), 0)
    divs = ring_divs[i]
    lines = []
    for j in range(divs):
        t1 = j / divs
        t2 = (j + 1) / divs
        p1 = Point(inner.x * (1 - t1) + outer.x * t1,
                   inner.y * (1 - t1) + outer.y * t1,
                   0)
        p2 = Point(inner.x * (1 - t2) + outer.x * t2,
                   inner.y * (1 - t2) + outer.y * t2,
                   0)
        lines.append(Line(p1, p2))
    radial_lines_dict[i] = lines

# Extract mesh nodes and connectivity
nodes = []
node_ids = {}
edges = []
fixed_nodes = set()

for i, lines in radial_lines_dict.items():
    for j, line in enumerate(lines):
        for pt in [line.start, line.end]:
            key = (round(pt.x, 8), round(pt.y, 8), round(pt.z, 8))
            if key not in node_ids:
                node_ids[key] = len(nodes)
                nodes.append(pt)
    for j, line in enumerate(lines):
        id1 = node_ids[(round(line.start.x, 8), round(line.start.y, 8), round(line.start.z, 8))]
        id2 = node_ids[(round(line.end.x, 8), round(line.end.y, 8), round(line.end.z, 8))]
        edges.append((id1, id2))

# Identify boundary nodes (based on radial ends)
for i, lines in radial_lines_dict.items():
    start_node = lines[0].start
    end_node = lines[-1].end
    for pt in [start_node, end_node]:
        key = (round(pt.x, 8), round(pt.y, 8), round(pt.z, 8))
        idx = node_ids[key]
        if abs(pt.x) == rectangle_radius or abs(pt.y) == rectangle_radius:
            fixed_nodes.add((idx, 'xyz'))  # corner fixed
        else:
            fixed_nodes.add((idx, 'z'))  # boundary ring fixed in Z

# Prep for dynamic relaxation
positions = np.array([[pt.x, pt.y, pt.z] for pt in nodes], dtype=np.float64)
velocities = np.zeros_like(positions)
forces = np.zeros_like(positions)
masses = np.ones(len(nodes))

# Dynamic relaxation loop parameters
dt = 0.01
damping = 0.98
EA = 1.0
tol = 1e-4
max_iters = 500

def dynamic_relaxation(positions, velocities, edges, fixed_nodes):
    for it in range(max_iters):
        forces[:] = 0
        # Gravity
        forces[:, 2] -= 1.0 * masses  # gravity in -Z

        # Internal spring forces
        for i1, i2 in edges:
            p1 = positions[i1]
            p2 = positions[i2]
            l0 = np.linalg.norm(p1 - p2)
            if l0 == 0:
                continue
            direction = (p2 - p1) / l0
            current_length = np.linalg.norm(positions[i2] - positions[i1])
            force = EA * (current_length - l0) * direction
            forces[i1] += force
            forces[i2] -= force

        # Update velocities
        velocities += (forces / masses[:, np.newaxis]) * dt
        velocities *= damping

        # Apply velocities to positions
        positions += velocities * dt

        # Apply constraints
        for idx, constraint in fixed_nodes:
            if 'x' in constraint:
                positions[idx, 0] = nodes[idx].x
                velocities[idx, 0] = 0
            if 'y' in constraint:
                positions[idx, 1] = nodes[idx].y
                velocities[idx, 1] = 0
            if 'z' in constraint:
                positions[idx, 2] = nodes[idx].z
                velocities[idx, 2] = 0

        # Check convergence
        max_force = np.max(np.linalg.norm(forces, axis=1))
        if max_force < tol:
            break
    return positions

final_positions = dynamic_relaxation(positions.copy(), velocities.copy(), edges, fixed_nodes)

# Visualise final mesh
x, y, z = [], [], []
for i1, i2 in edges:
    x += [final_positions[i1, 0], final_positions[i2, 0], None]
    y += [final_positions[i1, 1], final_positions[i2, 1], None]
    z += [final_positions[i1, 2], final_positions[i2, 2], None]

fig = go.Figure(data=go.Scatter3d(
    x=x, y=y, z=z,
    mode='lines',
    line=dict(color='blue', width=3)
))
fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectmode='data'
    ),
    title="Deformed Mesh After Dynamic Relaxation"
)
fig.show()


In [ ]:
import plotly.graph_objects as go

# Gather line coordinates from final_positions using the radial_lines_dict
x_coords, y_coords, z_coords = [], [], []
for i, sub_lines in radial_lines_dict.items():
    for line in sub_lines:
        start_idx = point_to_index.get(line.start)
        end_idx = point_to_index.get(line.end)
        if start_idx is None or end_idx is None:
            continue
        start_pos = final_positions[start_idx]
        end_pos = final_positions[end_idx]
        x_coords.extend([start_pos[0], end_pos[0], None])
        y_coords.extend([start_pos[1], end_pos[1], None])
        z_coords.extend([start_pos[2], end_pos[2], None])

# Create 3D line trace for deformed mesh
fig = go.Figure()

fig.add_trace(go.Scatter3d(
    x=x_coords,
    y=y_coords,
    z=z_coords,
    mode='lines',
    line=dict(color='blue', width=2),
    name='Deformed Mesh'
))

# Add boundary points for reference
for points, name, color in [(circle_points, 'Inner Boundary', 'red'), (rectangle_points, 'Outer Boundary', 'green')]:
    fig.add_trace(go.Scatter3d(
        x=[p.x for p in points],
        y=[p.y for p in points],
        z=[p.z for p in points],
        mode='markers',
        marker=dict(size=3, color=color),
        name=name
    ))

# Set layout
fig.update_layout(
    scene=dict(
        xaxis=dict(title='X'),
        yaxis=dict(title='Y'),
        zaxis=dict(title='Z'),
        aspectmode='data'
    ),
    title="Deformed Mesh After Dynamic Relaxation"
)

fig.show()
